In [1]:
import numpy as np
from Data_Savior_J import load_file

Xz = load_file("./classifier_data/walk1.data")
Xz = np.vstack((Xz,load_file("./classifier_data/walk1U.data")))
#Xz = np.vstack((Xz,load_file("./classifier_data/walk1D.data")))

Xz = np.vstack((Xz,load_file("./classifier_data/walk2.data")))
Xz = np.vstack((Xz,load_file("./classifier_data/walk2U.data")))
#Xz = np.vstack((Xz,load_file("./classifier_data/walk2D.data")))

Xz = np.vstack((Xz,load_file("./classifier_data/walk3.data")))
Xz = np.vstack((Xz,load_file("./classifier_data/walk3U.data")))
#Xz = np.vstack((Xz,load_file("./classifier_data/walk3D.data")))

Xz = np.vstack((Xz,load_file("./classifier_data/walk4.data")))
Xz = np.vstack((Xz,load_file("./classifier_data/walk4U.data")))
#Xz = np.vstack((Xz,load_file("./classifier_data/walk4D.data")))

Xz = np.vstack((Xz,load_file("./classifier_data/walk5.data")))
Xz = np.vstack((Xz,load_file("./classifier_data/walk5U.data")))
#Xz = np.vstack((Xz,load_file("./classifier_data/walk5D.data")))

## Vetor de features para classificação:

$X_c = [a~| ~av~| ~aa~| ~l\_a~| ~l\_av~| ~l\_aa~| ~pos\_foot\_r~| ~pos\_foot\_l~| ~vz\_r~| ~vz\_l~| ~C]$
$X_c = [0~|~~1~~| ~~2~~| ~~~3~~| ~~~4~~~| ~~~~5~~| ~~~~~~~6~~~~~~~~~~| ~~~~~~~~7~~~~~~~~| ~~~8~~~| ~~~~9~~| 10]$

#### $a \rightarrow$ ângulo do joelho direito; $av \rightarrow$ velocidade angular do joelho direito; $aa \rightarrow$ aceleração angular do joelho direito;
#### $a \rightarrow$ ângulo do joelho esquerdo; $av \rightarrow$ velocidade angular do joelho esquerdo; $aa \rightarrow$ aceleração angular do joelho esquerdo; 
#### $pos\_foot\_r \rightarrow$ posição do tornozelo direito em relação ao sacro; $pos\_foot\_l \rightarrow$ posição do tornozelo esquerdo em relação ao sacro;
#### $vz\_r \rightarrow$ velocidade do trocanter direito no eixo z; $vz\_l \rightarrow$ velocidade do trocanter esquerdo no eixo z;
#### $C \rightarrow$ indice de classificação
## Indice de classificação $"c"$:
#### $C = 0 \rightarrow$ Marcha normal;
#### $C = 1 \rightarrow$ Marcha de subida de escada;
#### $C = 2 \rightarrow$ Marvha de descidade escada.

In [2]:
import numpy as np

#X = Xz[:,[0,1,2,3,4,5,6,7]] #Utilizando as features distância entre tornozelos direto/esquerdo e sacro
X = Xz[:,[0,1,2,3,4,5,6,7,8,9]]

yz = Xz[:,[10]]
y = np.array([])
for i in range(len(yz)):
    y = np.hstack((y,yz[i]))

X.shape, y.shape

((550, 10), (550,))

In [3]:
np.unique(y) # possíveis valores de y

array([ 0.,  1.])

In [4]:
from sklearn.cross_validation import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, 
                                                    random_state=10)

In [154]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
sc.fit(X_train)
X_train_std = sc.transform(X_train)
X_test_std = sc.transform(X_test)


In [155]:
print X_train_std.shape, X_test.shape

(495, 10) (55, 10)


In [156]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn import svm, datasets

print 'Processing time RBF'
%time rbf_svc = svm.SVC(kernel='rbf', gamma=0.1, C=6, decision_function_shape='ovo').fit(X_train_std, y_train)
print ''
print 'Processing time Polynomial'
#%time poly_svc = svm.SVC(kernel='poly', degree=4, coef0=6.0, C=0.5).fit(X_train_std, y_train)
%time poly_svc = svm.SVC(kernel='poly', degree=2, coef0=4.5, C=48.9, decision_function_shape='ovr').fit(X_train_std, y_train)

Processing time RBF
CPU times: user 4 ms, sys: 0 ns, total: 4 ms
Wall time: 3.6 ms

Processing time Polynomial
CPU times: user 4 ms, sys: 0 ns, total: 4 ms
Wall time: 2.71 ms


In [157]:
def run_svm(svc, X_test_std, y_test):
    y_pred = svc.predict(X_test_std)
    from sklearn.metrics import accuracy_score
    if (svc==rbf_svc):
        print ('SVM-RBF accuracy:---------->%.2f %%' % (accuracy_score(y_test, y_pred)*100))
    elif(svc==poly_svc):
        print ('SVM-Polynomial accuracy:--->%.2f %%' % (accuracy_score(y_test, y_pred)*100))
        

In [158]:
run_svm(rbf_svc, X_test_std, y_test)
run_svm(poly_svc, X_test_std, y_test)


SVM-RBF accuracy:---------->100.00 %
SVM-Polynomial accuracy:--->100.00 %


In [128]:
from sklearn.grid_search import GridSearchCV
param_grid = [
    {
        'C'     : [0.001, 0.01, 0.05, 0.1, 0.5, 1, 1.5, 3, 6, 8,10, 38.5, 38.56, 38.8, 39, 39.01, 39.1, 39.2, 48.5, 48.56, 48.8, 49, 49.01, 49.1, 49.2, 50, 70, 80,
                   100, 1000], 
        'gamma' : [1000, 100, 80, 50, 35, 10, 7, 5, 3, 2, 1.5, 1, 0.9, 0.7, 0.8, 0.61, 0.62, 0.63, 0.65, 0.6, 0.59, 0.58, 0.57, 0.56, 
                   0.55, 0.5, 0.48, 0.45, 0.4, 0.35, 0.32, 0.25, 0.2, 0.1, 0.01, 0.001, 0.0001], 
        'kernel': ['rbf'],
        'random_state' : [1,5,10,20,30,40,50,60,70,80,90,100,500,1000,10000]
    },
]    
clf = GridSearchCV(svm.SVC(C=1), param_grid, cv=5)
%time clf.fit(X_train_std, y_train)

print("Best parameters set found on development set:")
print()
print(clf.best_params_)
print()
print("Grid scores on development set:")
print()
for params, mean_score, scores in clf.grid_scores_:
    print("%0.3f (+/-%0.03f) for %r"
          % (mean_score, scores.std() * 2, params))
print()


CPU times: user 7min 36s, sys: 108 ms, total: 7min 36s
Wall time: 7min 36s
Best parameters set found on development set:
()
{'kernel': 'rbf', 'C': 6, 'random_state': 1, 'gamma': 0.1}
()
Grid scores on development set:
()
0.527 (+/-0.005) for {'kernel': 'rbf', 'C': 0.001, 'random_state': 1, 'gamma': 1000}
0.527 (+/-0.005) for {'kernel': 'rbf', 'C': 0.001, 'random_state': 5, 'gamma': 1000}
0.527 (+/-0.005) for {'kernel': 'rbf', 'C': 0.001, 'random_state': 10, 'gamma': 1000}
0.527 (+/-0.005) for {'kernel': 'rbf', 'C': 0.001, 'random_state': 20, 'gamma': 1000}
0.527 (+/-0.005) for {'kernel': 'rbf', 'C': 0.001, 'random_state': 30, 'gamma': 1000}
0.527 (+/-0.005) for {'kernel': 'rbf', 'C': 0.001, 'random_state': 40, 'gamma': 1000}
0.527 (+/-0.005) for {'kernel': 'rbf', 'C': 0.001, 'random_state': 50, 'gamma': 1000}
0.527 (+/-0.005) for {'kernel': 'rbf', 'C': 0.001, 'random_state': 60, 'gamma': 1000}
0.527 (+/-0.005) for {'kernel': 'rbf', 'C': 0.001, 'random_state': 70, 'gamma': 1000}
0.527 (

In [60]:

def frange(start, stop, step):
    i = start
    while i < stop:
        yield i
        i += step
v = []
for i in frange(38.9, 39.1, 0.005):
     v.append(i)
%time v


CPU times: user 0 ns, sys: 0 ns, total: 0 ns
Wall time: 8.11 µs


[38.9,
 38.905,
 38.910000000000004,
 38.915000000000006,
 38.92000000000001,
 38.92500000000001,
 38.930000000000014,
 38.93500000000002,
 38.94000000000002,
 38.94500000000002,
 38.950000000000024,
 38.95500000000003,
 38.96000000000003,
 38.96500000000003,
 38.970000000000034,
 38.97500000000004,
 38.98000000000004,
 38.98500000000004,
 38.990000000000045,
 38.99500000000005,
 39.00000000000005,
 39.00500000000005,
 39.010000000000055,
 39.01500000000006,
 39.02000000000006,
 39.02500000000006,
 39.030000000000065,
 39.03500000000007,
 39.04000000000007,
 39.04500000000007,
 39.050000000000075,
 39.05500000000008,
 39.06000000000008,
 39.06500000000008,
 39.070000000000086,
 39.07500000000009,
 39.08000000000009,
 39.08500000000009,
 39.090000000000096,
 39.0950000000001]

In [5]:
%time
def frange(start, stop, step):
    i = start
    while i < stop:
        yield i
        i += step
g = []
for i in frange(0.5, 0.56, 0.0005):
     g.append(i)
g

CPU times: user 0 ns, sys: 0 ns, total: 0 ns
Wall time: 6.2 µs


[0.5,
 0.5005,
 0.5009999999999999,
 0.5014999999999998,
 0.5019999999999998,
 0.5024999999999997,
 0.5029999999999997,
 0.5034999999999996,
 0.5039999999999996,
 0.5044999999999995,
 0.5049999999999994,
 0.5054999999999994,
 0.5059999999999993,
 0.5064999999999993,
 0.5069999999999992,
 0.5074999999999992,
 0.5079999999999991,
 0.5084999999999991,
 0.508999999999999,
 0.509499999999999,
 0.5099999999999989,
 0.5104999999999988,
 0.5109999999999988,
 0.5114999999999987,
 0.5119999999999987,
 0.5124999999999986,
 0.5129999999999986,
 0.5134999999999985,
 0.5139999999999985,
 0.5144999999999984,
 0.5149999999999983,
 0.5154999999999983,
 0.5159999999999982,
 0.5164999999999982,
 0.5169999999999981,
 0.5174999999999981,
 0.517999999999998,
 0.518499999999998,
 0.5189999999999979,
 0.5194999999999979,
 0.5199999999999978,
 0.5204999999999977,
 0.5209999999999977,
 0.5214999999999976,
 0.5219999999999976,
 0.5224999999999975,
 0.5229999999999975,
 0.5234999999999974,
 0.5239999999999974,
 0